# MONTAR O DRIVE

In [1]:
# MONTAR O DRIVE

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DOWNLOAD DOS TICKERS EM ANÁLISE

In [ ]:
# ACK HEURISTICS: HF000=ON | GapFill=OFF | ContextCarry=OFF | Narrative=OFF | StructRetain=OFF | SpeedOpt=OFF | Mode=Instant | ExecProfile={RAM_ONLY}
def execute_data_download_dl24_yf_20120101():
    """
    Auto-contained function to download financial data as per DL24_YF_20120101 instructions.
    This function will handle library installation, drive mounting, data fetching with retries,
    and saving data and metadata, respecting the dry_run flag.
    """
    # Rodada A: dry_run=True (simular, sem escrever).
    # Para Rodada B, mude para dry_run=False após receber o "GO".
    dry_run = False

    # --- BOOTSTRAP ---
    try:
        import yfinance
    except ImportError:
        print("yfinance not found. Installing...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "yfinance", "--quiet"])
        import yfinance
    # --- END BOOTSTRAP ---

    import os
    import time
    import json
    import pandas as pd
    from datetime import datetime
    import traceback
    from google.colab import drive

    # --- CONFIGURATION (SSOT) ---
    TICKERS = [
        "ITUB4.SA", "BBAS3.SA", "B3SA3.SA", "PSSA3.SA",
        "VALE3.SA", "GGBR4.SA", "CSNA3.SA", "SUZB3.SA",
        "PETR4.SA", "PRIO3.SA", "UGPA3.SA",
        "ELET3.SA", "TAEE11.SA", "CPLE6.SA",
        "SBSP3.SA",
        "VIVT3.SA", "TIMS3.SA",
        "RDOR3.SA", "HAPV3.SA",
        "ABEV3.SA", "WEGE3.SA", "TOTS3.SA", "LREN3.SA",
        "RAIL3.SA"
    ]
    ROOT_PATH = "/content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/"
    DESTINATION_SUBDIR = "00_data/01_raw/"
    DESTINATION_PATH = os.path.join(ROOT_PATH, DESTINATION_SUBDIR)

    START_DATE = "2012-01-01"
    END_DATE = datetime.now().strftime('%Y-%m-%d')
    TODAY_SUFFIX = datetime.now().strftime('%Y%m%d')

    MAX_RETRIES = 3
    BACKOFF_SECONDS = 5

    # --- EXECUTION ---
    print(f"--- INICIANDO EXECUÇÃO DL24_YF_20120101 ---")
    print(f"Modo de Execução: {'DRY RUN (Simulação)' if dry_run else 'PERSIST (Gravação Real)'}")
    print("-" * 50)

    # 1. Montar Google Drive e Validar Caminhos
    try:
        print("Montando Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("Drive montado com sucesso.")

        if not os.path.isdir(ROOT_PATH):
            print(f"ERRO CRÍTICO: O caminho ROOT '{ROOT_PATH}' não foi encontrado. Verifique as permissões e o nome do Shared Drive.")
            return

        if dry_run:
            print(f"DRY RUN: Validaria a existência ou criaria o diretório de destino: {DESTINATION_PATH}")
        else:
            print(f"Verificando/Criando diretório de destino: {DESTINATION_PATH}")
            os.makedirs(DESTINATION_PATH, exist_ok=True)
        print("Validação de caminhos concluída.")
    except Exception as e:
        print(f"Falha crítica ao montar o Drive ou validar caminhos. Encerrando.")
        print(traceback.format_exc())
        return

    processed_tickers = []
    failed_tickers = []

    # 2. Loop de Processamento por Ticker
    for ticker in TICKERS:
        print(f"\n--- Processando ticker: {ticker} ---")
        success = False
        last_exception = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                print(f"Tentativa {attempt}/{MAX_RETRIES} para {ticker}...")

                # Download de dados
                yf_ticker = yfinance.Ticker(ticker)

                history_df = yf_ticker.history(
                    start=START_DATE,
                    end=END_DATE,
                    interval="1d",
                    actions=True,
                    auto_adjust=False # Crucial para manter Adj Close e colunas de ações
                )

                if history_df.empty:
                    raise ValueError(f"Nenhum dado retornado para o ticker {ticker} no período especificado.")

                # Extração de Metadados
                metadata = {}
                try:
                    # fast_info é mais rápido e geralmente suficiente
                    metadata = yf_ticker.fast_info.to_dict() if hasattr(yf_ticker, 'fast_info') else {}
                    if not metadata: # Fallback para 'info' se fast_info estiver vazio ou não existir
                        metadata = yf_ticker.info if hasattr(yf_ticker, 'info') else {}
                except Exception as meta_e:
                    print(f"AVISO: Não foi possível obter metadados para {ticker}. Erro: {meta_e}")

                # Geração de nomes de arquivo
                ticker_lower = ticker.lower().replace('.sa', '_sa')
                ohlcv_filename = f"{ticker_lower}_ohlcv_actions_20120101_{TODAY_SUFFIX}.parquet"
                metadata_filename = f"{ticker_lower}_metadata_{TODAY_SUFFIX}.json"

                ohlcv_filepath = os.path.join(DESTINATION_PATH, ohlcv_filename)
                metadata_filepath = os.path.join(DESTINATION_PATH, metadata_filename)

                # --- LOGS MÍNIMOS ---
                print(f"Ticker: {ticker}")
                print(f"  - Linhas totais: {len(history_df)}")
                if not history_df.empty:
                    print(f"  - Date Min: {history_df.index.min().strftime('%Y-%m-%d')}")
                    print(f"  - Date Max: {history_df.index.max().strftime('%Y-%m-%d')}")

                required_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits']
                col_presence = {col: (col in history_df.columns) for col in required_cols}
                print(f"  - Presença de colunas: {col_presence}")

                # --- PERSISTÊNCIA (ou Simulação) ---
                # Salvar OHLCV
                if dry_run:
                    print(f"  - DRY RUN: Simularia salvar OHLCV em: {ohlcv_filepath}")
                elif os.path.exists(ohlcv_filepath):
                    print(f"  - AVISO: Arquivo OHLCV já existe, pulando: {ohlcv_filepath}")
                else:
                    history_df.to_parquet(ohlcv_filepath)
                    print(f"  - SUCESSO: OHLCV salvo em: {ohlcv_filepath}")

                # Salvar Metadados
                if metadata:
                    if dry_run:
                        print(f"  - DRY RUN: Simularia salvar Metadados em: {metadata_filepath}")
                    elif os.path.exists(metadata_filepath):
                        print(f"  - AVISO: Arquivo de metadados já existe, pulando: {metadata_filepath}")
                    else:
                        with open(metadata_filepath, 'w', encoding='utf-8') as f:
                            json.dump(metadata, f, ensure_ascii=False, indent=4)
                        print(f"  - SUCESSO: Metadados salvos em: {metadata_filepath}")
                else:
                    print(f"  - AVISO: Nenhum metadado para salvar para {ticker}.")

                processed_tickers.append(ticker)
                success = True
                break  # Sai do loop de tentativas se obteve sucesso

            except Exception as e:
                print(f"ERRO na tentativa {attempt} para {ticker}: {e}")
                last_exception = traceback.format_exc()
                if attempt < MAX_RETRIES:
                    print(f"Aguardando {BACKOFF_SECONDS} segundos antes da próxima tentativa...")
                    time.sleep(BACKOFF_SECONDS)

        if not success:
            failed_tickers.append({"ticker": ticker, "traceback": last_exception})

    # --- RELATÓRIO FINAL ---
    print("\n" + "=" * 50)
    print("RELATÓRIO FINAL DA EXECUÇÃO")
    print(f"Modo: {'DRY RUN (Simulação)' if dry_run else 'PERSIST (Gravação Real)'}")
    print("=" * 50)
    print(f"Tickers processados com sucesso ({len(processed_tickers)}/{len(TICKERS)}):")
    print(processed_tickers if processed_tickers else "Nenhum.")
    print("-" * 50)
    if failed_tickers:
        print(f"Tickers que falharam ({len(failed_tickers)}/{len(TICKERS)}):")
        for failure in failed_tickers:
            print(f"\n--- Ticker: {failure['ticker']} ---")
            print(failure['traceback'])
    else:
        print("Nenhuma falha registrada.")
    print("=" * 50)
    print("--- EXECUÇÃO DL24_YF_20120101 CONCLUÍDA ---")

# Invocar a função principal
execute_data_download_dl24_yf_20120101()

--- INICIANDO EXECUÇÃO DL24_YF_20120101 ---
Modo de Execução: PERSIST (Gravação Real)
--------------------------------------------------
Montando Google Drive...
Mounted at /content/drive
Drive montado com sucesso.
Verificando/Criando diretório de destino: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/
Validação de caminhos concluída.

--- Processando ticker: ITUB4.SA ---
Tentativa 1/3 para ITUB4.SA...
AVISO: Não foi possível obter metadados para ITUB4.SA. Erro: 'FastInfo' object has no attribute 'to_dict'
Ticker: ITUB4.SA
  - Linhas totais: 3409
  - Date Min: 2012-01-02
  - Date Max: 2025-09-19
  - Presença de colunas: {'Open': True, 'High': True, 'Low': True, 'Close': True, 'Adj Close': True, 'Volume': True, 'Dividends': True, 'Stock Splits': True}
  - SUCESSO: OHLCV salvo em: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/itub4_sa_ohlcv_actions_20120101_20250922.parquet
  - AVISO: Nenhum metadado para salvar para ITUB4.SA.

--- Pr

# DOWNLOAD DOS MARCADORES (IBOVESA, BRENT, ETC..)

In [ ]:
# ACK HEURISTICS: HF000=ON | GapFill=OFF | ContextCarry=OFF | Narrative=OFF | StructRetain=OFF | SpeedOpt=OFF | Mode=Instant | ExecProfile={RAM_ONLY}
def execute_macro_data_download_dl_macros_20120101():
    """
    Auto-contained function to download macroeconomic data as per DL_MACROS_20120101 instructions.
    Handles library installation, drive mounting, data fetching with retries and fallbacks,
    and saving data and metadata with versioning, respecting the dry_run flag.
    """
    # Rodada A: dry_run=True (simular, sem escrever).
    # Para Rodada B, mude para dry_run=False após receber o "GO".
    dry_run = False

    # --- BOOTSTRAP ---
    try:
        import yfinance
    except ImportError:
        print("yfinance not found. Installing...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "yfinance", "--quiet"])
        import yfinance
    # --- END BOOTSTRAP ---

    import os
    import time
    import json
    import pandas as pd
    from datetime import datetime
    import traceback
    from google.colab import drive
    import re

    # --- CONFIGURATION (SSOT) ---
    ASSETS = [
        "^BVSP",  # Ibovespa – índice âncora
        "EWZ",    # iShares MSCI Brazil ETF
        "^GSPC",  # S&P 500
        "^VIX",   # CBOE Volatility Index
        "DX-Y.NYB", # US Dollar Index – DXY
        "^TNX",   # US 10Y Treasury Yield
        "BZ=F"    # Brent crude oil futures
    ]
    FALLBACK_MAP = {
        "BZ=F": "CL=F"  # WTI as fallback for Brent
    }

    ROOT_PATH = "/content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/"
    DESTINATION_SUBDIR = "00_data/01_raw/"
    DESTINATION_PATH = os.path.join(ROOT_PATH, DESTINATION_SUBDIR)

    START_DATE = "2012-01-01"
    END_DATE = datetime.now().strftime('%Y-%m-%d')
    TODAY_SUFFIX = datetime.now().strftime('%Y%m%d')

    MAX_RETRIES = 3
    BACKOFF_SECONDS = 5

    # --- HELPER FUNCTIONS ---
    def sanitize_symbol_for_filename(symbol):
        """Sanitizes a ticker symbol to be used in a filename."""
        return re.sub(r'[^a-zA-Z0-9=\-_\.]', '_', symbol).lower()

    def get_unique_filepath(filepath):
        """Checks if a file exists and returns a versioned path if it does."""
        if not os.path.exists(filepath):
            return filepath

        base, ext = os.path.splitext(filepath)
        version = 2
        while True:
            new_filepath = f"{base}_v{version}{ext}"
            if not os.path.exists(new_filepath):
                return new_filepath
            version += 1

    # --- EXECUTION ---
    print(f"--- INICIANDO EXECUÇÃO DL_MACROS_20120101 ---")
    print(f"Modo de Execução: {'DRY RUN (Simulação)' if dry_run else 'PERSIST (Gravação Real)'}")
    print("-" * 50)

    # 1. Montar Google Drive e Validar Caminhos
    try:
        print("Montando Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("Drive montado com sucesso.")

        if not os.path.isdir(ROOT_PATH):
            print(f"ERRO CRÍTICO: O caminho ROOT '{ROOT_PATH}' não foi encontrado.")
            return

        if dry_run:
            print(f"DRY RUN: Validaria a existência ou criaria o diretório de destino: {DESTINATION_PATH}")
        else:
            print(f"Verificando/Criando diretório de destino: {DESTINATION_PATH}")
            os.makedirs(DESTINATION_PATH, exist_ok=True)
        print("Validação de caminhos concluída.")
    except Exception as e:
        print(f"Falha crítica ao montar o Drive ou validar caminhos. Encerrando.")
        print(traceback.format_exc())
        return

    processed_assets = []
    failed_assets = []

    # 2. Loop de Processamento por Ativo
    for asset in ASSETS:
        print(f"\n--- Processando ativo: {asset} ---")
        success = False
        last_exception = None
        asset_to_download = asset
        is_fallback = False

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                print(f"Tentativa {attempt}/{MAX_RETRIES} para '{asset_to_download}' (alvo original: '{asset}')")

                yf_ticker = yfinance.Ticker(asset_to_download)

                history_df = yf_ticker.history(
                    start=START_DATE,
                    end=END_DATE,
                    interval="1d",
                    actions=True,
                    auto_adjust=False
                )

                if history_df.empty:
                    raise ValueError(f"Nenhum dado retornado para '{asset_to_download}' no período.")

                metadata = {}
                try:
                    metadata = yf_ticker.info if hasattr(yf_ticker, 'info') else {}
                except Exception as meta_e:
                    print(f"AVISO: Não foi possível obter metadados para {asset_to_download}. Erro: {meta_e}")

                # Se chegou aqui, o download foi bem-sucedido
                sanitized_original_asset = sanitize_symbol_for_filename(asset)
                ohlcv_filename_base = f"{sanitized_original_asset}_ohlcv_actions_20120101_{TODAY_SUFFIX}.parquet"
                metadata_filename_base = f"{sanitized_original_asset}_metadata_{TODAY_SUFFIX}.json"

                ohlcv_filepath = os.path.join(DESTINATION_PATH, ohlcv_filename_base)
                metadata_filepath = os.path.join(DESTINATION_PATH, metadata_filename_base)

                # --- VERBOSITY=REPORT ---
                if is_fallback:
                    print(f"  - ALERTA: Utilizado fallback. Ativo original: '{asset}', Baixado: '{asset_to_download}' (alias: WTI_fallback_for_Brent)")
                print(f"  - Linhas totais: {len(history_df)}")
                if not history_df.empty:
                    print(f"  - Date Min: {history_df.index.min().strftime('%Y-%m-%d')}")
                    print(f"  - Date Max: {history_df.index.max().strftime('%Y-%m-%d')}")

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                col_presence = {col: (col in history_df.columns) for col in required_cols}
                print(f"  - Presença de colunas: {col_presence}")

                # --- PERSISTÊNCIA (ou Simulação) com versionamento ---
                final_ohlcv_path = get_unique_filepath(ohlcv_filepath) if not dry_run else ohlcv_filepath
                final_metadata_path = get_unique_filepath(metadata_filepath) if not dry_run else metadata_filepath

                if dry_run:
                    print(f"  - DRY RUN: Simularia salvar OHLCV em: {get_unique_filepath(ohlcv_filepath)}")
                    if metadata:
                        print(f"  - DRY RUN: Simularia salvar Metadados em: {get_unique_filepath(metadata_filepath)}")
                else:
                    history_df.to_parquet(final_ohlcv_path)
                    print(f"  - SUCESSO: OHLCV salvo em: {final_ohlcv_path}")
                    if metadata:
                        with open(final_metadata_path, 'w', encoding='utf-8') as f:
                            json.dump(metadata, f, ensure_ascii=False, indent=4)
                        print(f"  - SUCESSO: Metadados salvos em: {final_metadata_path}")

                processed_assets.append(asset)
                success = True
                break

            except Exception as e:
                print(f"ERRO na tentativa {attempt} para '{asset_to_download}': {e}")
                last_exception = traceback.format_exc()

                # Lógica de Fallback
                if asset_to_download in FALLBACK_MAP and attempt == 1: # Tenta o fallback na segunda tentativa
                    fallback_asset = FALLBACK_MAP[asset_to_download]
                    print(f"INFO: Ativando fallback para '{asset}'. Tentando com '{fallback_asset}'...")
                    asset_to_download = fallback_asset
                    is_fallback = True
                    continue # Pula o sleep e vai direto para a próxima tentativa com o fallback

                if attempt < MAX_RETRIES:
                    print(f"Aguardando {BACKOFF_SECONDS} segundos...")
                    time.sleep(BACKOFF_SECONDS)

        if not success:
            failed_assets.append({"asset": asset, "traceback": last_exception})

    # --- RELATÓRIO FINAL ---
    print("\n" + "=" * 50)
    print("RELATÓRIO FINAL DA EXECUÇÃO")
    print(f"Modo: {'DRY RUN (Simulação)' if dry_run else 'PERSIST (Gravação Real)'}")
    print("=" * 50)
    print(f"Ativos processados com sucesso ({len(processed_assets)}/{len(ASSETS)}):")
    print(processed_assets if processed_assets else "Nenhum.")
    print("-" * 50)
    if failed_assets:
        print(f"Ativos que falharam ({len(failed_assets)}/{len(ASSETS)}):")
        for failure in failed_assets:
            print(f"\n--- Ativo: {failure['asset']} ---")
            print(failure['traceback'])
    else:
        print("Nenhuma falha registrada.")
    print("=" * 50)
    print("--- EXECUÇÃO DL_MACROS_20120101 CONCLUÍDA ---")

# Invocar a função principal
execute_macro_data_download_dl_macros_20120101()

--- INICIANDO EXECUÇÃO DL_MACROS_20120101 ---
Modo de Execução: PERSIST (Gravação Real)
--------------------------------------------------
Montando Google Drive...
Mounted at /content/drive
Drive montado com sucesso.
Verificando/Criando diretório de destino: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/
Validação de caminhos concluída.

--- Processando ativo: ^BVSP ---
Tentativa 1/3 para '^BVSP' (alvo original: '^BVSP')
  - Linhas totais: 3400
  - Date Min: 2012-01-03
  - Date Max: 2025-09-19
  - Presença de colunas: {'Open': True, 'High': True, 'Low': True, 'Close': True, 'Volume': True}
  - SUCESSO: OHLCV salvo em: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/_bvsp_ohlcv_actions_20120101_20250922.parquet
  - SUCESSO: Metadados salvos em: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/_bvsp_metadata_20250922.json

--- Processando ativo: EWZ ---
Tentativa 1/3 para 'EWZ' (alvo original: 'EWZ')
  - Linhas t

# MANIFESTO DOS DADOS BAIXADOS - CAMADA BRONZE


In [ ]:
# ACK HEURISTICS: HF000=ON | GapFill=OFF | ContextCarry=OFF | Narrative=OFF | StructRetain=OFF | SpeedOpt=OFF | Mode=Instant | ExecProfile={RAM_ONLY}
def generate_raw_manifest_v1():
    """
    Auto-contained function to generate a manifest of all Parquet files in the raw data directory,
    as per instruction RAW_MANIFEST_V1.
    """
    # Rodada A: dry_run=True (simular, sem gravar).
    # Para Rodada B, mude para dry_run=False após receber o "GO".
    dry_run = False

    import os
    import pandas as pd
    import hashlib
    from datetime import datetime
    import traceback
    from google.colab import drive

    # --- CONFIGURATION (SSOT) ---
    SCAN_PATH = "/content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/"
    TODAY_SUFFIX = datetime.now().strftime('%Y%m%d')
    OUTPUT_FILENAME_BASE = f"raw_manifest_{TODAY_SUFFIX}.csv"

    # --- HELPER FUNCTIONS ---
    def get_unique_filepath(filepath):
        """Checks if a file exists and returns a versioned path if it does."""
        if not os.path.exists(filepath) and not dry_run:
            return filepath

        # In dry_run, we always want to show the base path unless it already exists,
        # then we show the next available versioned path.
        if os.path.exists(filepath):
            base, ext = os.path.splitext(filepath)
            version = 2
            while True:
                new_filepath = f"{base}_v{version}{ext}"
                if not os.path.exists(new_filepath):
                    return new_filepath
                version += 1
        return filepath


    def calculate_partial_hashes(file_path):
        """Calculates SHA256 hash of the first and last 20 bytes of a file."""
        sha256_head = hashlib.sha256()
        sha256_tail = hashlib.sha256()
        try:
            with open(file_path, 'rb') as f:
                # Head hash
                head_chunk = f.read(20)
                sha256_head.update(head_chunk)

                # Tail hash
                file_size = os.path.getsize(file_path)
                if file_size > 20:
                    f.seek(-20, os.SEEK_END)
                    tail_chunk = f.read(20)
                    sha256_tail.update(tail_chunk)
                else: # If file is 20 bytes or less, head and tail are the same
                    sha256_tail.update(head_chunk)

            return sha256_head.hexdigest(), sha256_tail.hexdigest()
        except Exception:
            return None, None

    # --- EXECUTION ---
    print(f"--- INICIANDO EXECUÇÃO RAW_MANIFEST_V1 ---")
    print(f"Modo de Execução: {'DRY RUN (Simulação)' if dry_run else 'PERSIST (Gravação Real)'}")
    print("-" * 50)

    # 1. Montar Google Drive e Validar Caminho
    try:
        print("Montando Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("Drive montado com sucesso.")
        if not os.path.isdir(SCAN_PATH):
            print(f"ERRO CRÍTICO: O caminho de varredura '{SCAN_PATH}' não foi encontrado.")
            return
        print(f"Caminho de varredura validado: {SCAN_PATH}")
    except Exception as e:
        print(f"Falha crítica ao montar o Drive ou validar caminhos. Encerrando.")
        print(traceback.format_exc())
        return

    # 2. Varredura de Arquivos Parquet
    parquet_files = []
    for root, _, files in os.walk(SCAN_PATH):
        for file in files:
            if file.endswith('.parquet'):
                parquet_files.append(os.path.join(root, file))

    if not parquet_files:
        print("Nenhum arquivo Parquet encontrado. Encerrando.")
        return

    # 3. Processamento e Geração do Manifesto
    manifest_data = []
    required_cols = [
        'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
        'Dividends', 'Stock Splits'
    ]

    for file_path in parquet_files:
        try:
            file_info = {}
            file_name = os.path.basename(file_path)

            # Extrair ticker/symbol do nome do arquivo (e.g., 'itub4_sa_...')
            file_info['ticker_or_symbol'] = '_'.join(file_name.split('_')[:-3])
            file_info['file_name'] = file_name
            file_info['file_path'] = file_path

            # Ler metadados do Parquet
            df = pd.read_parquet(file_path)

            file_info['rows_total'] = len(df)
            if not df.empty and isinstance(df.index, pd.DatetimeIndex):
                file_info['date_min'] = df.index.min().strftime('%Y-%m-%d')
                file_info['date_max'] = df.index.max().strftime('%Y-%m-%d')
            else:
                file_info['date_min'] = None
                file_info['date_max'] = None

            # Verificar presença de colunas (Anti-ambiguidade)
            # 'Adj Close' is a special case from yfinance history
            file_info['has_AdjClose'] = 'Adj Close' in df.columns
            file_info['has_StockSplits'] = 'Stock Splits' in df.columns
            for col in ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends']:
                file_info[f'has_{col}'] = col in df.columns

            # Metadados do sistema de arquivos
            stat = os.stat(file_path)
            file_info['created_ts'] = datetime.fromtimestamp(stat.st_ctime).isoformat()
            file_info['modified_ts'] = datetime.fromtimestamp(stat.st_mtime).isoformat()
            file_info['file_size_bytes'] = stat.st_size

            # Hashes
            head_hash, tail_hash = calculate_partial_hashes(file_path)
            file_info['sha256_head20'] = head_hash
            file_info['sha256_tail20'] = tail_hash

            manifest_data.append(file_info)

        except Exception as e:
            print(f"\nERRO: Falha ao processar o arquivo: {file_path}")
            print(traceback.format_exc())
            continue

    # 4. Criar DataFrame Final
    manifest_df = pd.DataFrame(manifest_data)
    # Reordenar colunas para corresponder à instrução
    column_order = [
        'ticker_or_symbol', 'file_name', 'file_path', 'rows_total', 'date_min', 'date_max',
        'has_Open', 'has_High', 'has_Low', 'has_Close', 'has_AdjClose', 'has_Volume',
        'has_Dividends', 'has_StockSplits', 'created_ts', 'modified_ts',
        'file_size_bytes', 'sha256_head20', 'sha256_tail20'
    ]
    # Apenas reordena se o dataframe não estiver vazio
    if not manifest_df.empty:
        manifest_df = manifest_df[column_order]

    # --- RELATÓRIO E SAÍDA ---
    print("\n" + "=" * 50)
    print("RELATÓRIO DA GERAÇÃO DO MANIFESTO")
    print("=" * 50)
    print(f"Total de arquivos Parquet mapeados: {len(manifest_df)}")
    print("\nPrimeiras 5 linhas do manifesto gerado:")
    print(manifest_df.head())
    print("-" * 50)

    # 5. Salvar (ou simular) o arquivo CSV
    output_path = os.path.join(SCAN_PATH, OUTPUT_FILENAME_BASE)
    final_output_path = get_unique_filepath(output_path)

    if dry_run:
        print(f"DRY RUN: O manifesto seria salvo em: {final_output_path}")
    else:
        try:
            manifest_df.to_csv(final_output_path, index=False, encoding='utf-8')
            print(f"SUCESSO: Manifesto salvo em: {final_output_path}")
        except Exception as e:
            print(f"ERRO CRÍTICO: Falha ao salvar o arquivo CSV em '{final_output_path}'")
            print(traceback.format_exc())

    print("=" * 50)
    print("--- EXECUÇÃO RAW_MANIFEST_V1 CONCLUÍDA ---")

# Invocar a função principal
generate_raw_manifest_v1()

--- INICIANDO EXECUÇÃO RAW_MANIFEST_V1 ---
Modo de Execução: PERSIST (Gravação Real)
--------------------------------------------------
Montando Google Drive...
Mounted at /content/drive
Drive montado com sucesso.
Caminho de varredura validado: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/

RELATÓRIO DA GERAÇÃO DO MANIFESTO
Total de arquivos Parquet mapeados: 31

Primeiras 5 linhas do manifesto gerado:
  ticker_or_symbol                                         file_name  \
0   itub4_sa_ohlcv  itub4_sa_ohlcv_actions_20120101_20250922.parquet   
1   bbas3_sa_ohlcv  bbas3_sa_ohlcv_actions_20120101_20250922.parquet   
2   b3sa3_sa_ohlcv  b3sa3_sa_ohlcv_actions_20120101_20250922.parquet   
3   pssa3_sa_ohlcv  pssa3_sa_ohlcv_actions_20120101_20250922.parquet   
4   vale3_sa_ohlcv  vale3_sa_ohlcv_actions_20120101_20250922.parquet   

                                           file_path  rows_total    date_min  \
0  /content/drive/Shareddrives/BOLSA_2026/a_bolsa... 

# TRANSFORMAÇÃO EM SILVER E MANIFESTO

In [4]:
# ACK HEURISTICS: HF000=ON | GapFill=OFF | ContextCarry=OFF | Narrative=OFF | StructRetain=OFF | SpeedOpt=OFF | Mode=Instant | ExecProfile=PERSIST
def execute_silver_persist_and_manifest_v1():
    """
    Auto-contained function to persist the Silver layer artifacts and then generate
    a canonical manifest for the entire Silver directory, as per instruction
    SILVER_PERSIST_E_MANIFEST_V1. This is a direct execution with dry_run=False.
    """
    # --- BOOTSTRAP ---
    try:
        import pandas
        import pyarrow
    except ImportError:
        print("Required libraries not found. Installing pandas and pyarrow...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "pyarrow", "--quiet"])
    # --- END BOOTSTRAP ---

    import os
    import pandas as pd
    import hashlib
    import time
    from datetime import datetime
    import traceback

    # --- CONFIGURATION (SSOT) ---
    ROOT_PATH = "/content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/"
    RAW_PATH = os.path.join(ROOT_PATH, "00_data/01_raw/")
    SILVER_PATH = os.path.join(ROOT_PATH, "00_data/02_silver/")
    TODAY_SUFFIX = datetime.now().strftime('%Y%m%d')
    MACRO_ALIASES = ['_bvsp', 'ewz', '_gspc', '_vix', 'dx-y.nyb', '_tnx', 'bz=f']
    METRICS = {
        'close': 'Close', 'adj_close': 'Adj Close', 'volume': 'Volume',
        'dividends': 'Dividends', 'splits': 'Stock Splits'
    }

    # --- HELPER FUNCTIONS ---
    def get_unique_filepath(filepath):
        if not os.path.exists(filepath): return filepath
        base, ext = os.path.splitext(filepath)
        version = 2
        while True:
            new_filepath = f"{base}_v{version}{ext}";
            if not os.path.exists(new_filepath): return new_filepath
            version += 1

    def calculate_partial_hashes(file_path):
        sha256_head, sha256_tail = hashlib.sha256(), hashlib.sha256()
        try:
            with open(file_path, 'rb') as f:
                head_chunk = f.read(20); sha256_head.update(head_chunk)
                file_size = os.path.getsize(file_path)
                if file_size > 20: f.seek(-20, os.SEEK_END); sha256_tail.update(f.read(20))
                else: sha256_tail.update(head_chunk)
            return sha256_head.hexdigest(), sha256_tail.hexdigest()
        except Exception: return None, None

    def classify_symbol(alias):
        alias_lower = alias.lower()
        if alias_lower.endswith('_sa'): return 'equity_br'
        if alias_lower == 'ewz': return 'etf'
        if '=' in alias_lower: return 'futures'
        if alias_lower.startswith('_'): return 'index'
        return 'unknown'

    # --- EXECUTION ---
    print(f"--- INICIANDO EXECUÇÃO: SILVER_PERSIST_E_MANIFEST_V1 ---")
    print(f"Modo de Execução: PERSIST (Gravação Real)")
    print("-" * 50)

    # 0. Montagem e Validação de Caminhos
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        if not os.path.isdir(RAW_PATH): raise FileNotFoundError(f"Caminho RAW não encontrado: {RAW_PATH}")
        os.makedirs(SILVER_PATH, exist_ok=True)
        print(f"Caminhos validados.\nRAW: {RAW_PATH}\nSILVER: {SILVER_PATH}")
    except Exception as e:
        print(f"DÚVIDA_BLOQUEANTE: Falha crítica na montagem do Drive ou criação de diretórios: {e}\n{traceback.format_exc()}"); return

    # =========================================================================
    # PARTE A — Persistência Silver
    # =========================================================================
    print("\n" + "="*25 + " PARTE A: Persistência Silver " + "="*25)

    # 1. Descoberta e Preparação
    raw_parquet_files = [os.path.join(RAW_PATH, f) for f in os.listdir(RAW_PATH) if f.endswith('.parquet')]
    print(f"Encontrados {len(raw_parquet_files)} arquivos Parquet na camada RAW.")
    if len(raw_parquet_files) == 0:
        print("DÚVIDA_BLOQUEANTE: Nenhum arquivo Parquet encontrado na camada RAW. Abortando."); return

    alias_map_data = [{'file_name': os.path.basename(f), 'alias': os.path.basename(f).split('_ohlcv_')[0]} for f in raw_parquet_files]
    map_symbol_alias_df = pd.DataFrame(alias_map_data)

    # 2. Calendário Canônico B3 (com normalização robusta)
    print("\n--- Construindo Calendário Canônico B3 ---")
    b3_files = [os.path.join(RAW_PATH, f['file_name']) for _, f in map_symbol_alias_df.iterrows() if f['alias'].endswith('_sa')]
    all_b3_dates = set()
    for file_path in b3_files:
        try: all_b3_dates.update(pd.read_parquet(file_path, columns=[]).index)
        except: pass
    if not all_b3_dates: print("DÚVIDA_BLOQUEANTE: Não foi possível construir o calendário B3."); return
    calendar_b3 = pd.to_datetime(pd.to_datetime(sorted(list(all_b3_dates))).date)
    print(f"Calendário Canônico B3 construído: {len(calendar_b3)} dias, de {calendar_b3.min().date()} a {calendar_b3.max().date()}.")

    # 3. Leitura, Alinhamento e Montagem dos Artefatos
    print("\n--- Lendo, Alinhando e Montando Artefatos Silver ---")
    series_by_metric = {key: [] for key in METRICS}; dim_symbols_data = []
    for _, row in map_symbol_alias_df.iterrows():
        try:
            df = pd.read_parquet(os.path.join(RAW_PATH, row['file_name']))
            df.index = pd.to_datetime(pd.to_datetime(df.index).date) # Normalização robusta
            dim_symbols_data.append({
                'alias': row['alias'], 'class': classify_symbol(row['alias']),
                'date_min': df.index.min().date() if not df.empty else None,
                'date_max': df.index.max().date() if not df.empty else None,
                'rows_total_raw': df.shape[0]
            })
            for key, col_name in METRICS.items():
                if col_name in df.columns: series_by_metric[key].append(df[col_name].copy().rename(row['alias']).reindex(calendar_b3))
        except Exception as e: print(f"AVISO: Falha ao processar {row['file_name']}: {e}")

    tabeloes = {key: pd.concat(s_list, axis=1).sort_index(axis=1) for key, s_list in series_by_metric.items() if s_list}
    dim_symbols_df = pd.DataFrame(dim_symbols_data).sort_values('alias').reset_index(drop=True)

    # 4. Gravação, Validação e Relatórios
    print("\n--- Gravando e Validando Artefatos Silver ---")
    artefacts_to_save = {
        **{f"silver_{key}_{TODAY_SUFFIX}.parquet": df for key, df in tabeloes.items()},
        f"dim_symbols_{TODAY_SUFFIX}.parquet": dim_symbols_df,
        f"map_symbol_alias.csv": map_symbol_alias_df
    }
    saved_files = []

    for filename, df in artefacts_to_save.items():
        print("\n" + "-"*20); print(f"Processando: {filename}")

        # Validação pré-gravação
        is_tabelao = 'silver_' in filename and filename.endswith('.parquet')
        if is_tabelao:
            if df.shape[0] == 0 or df.shape[1] < 31:
                print(f"DÚVIDA_BLOQUEANTE: O tabelão '{filename}' falhou na validação de shape: {df.shape}. Esperado: rows > 0, cols >= 31. Abortando."); return
            for alias in MACRO_ALIASES:
                if alias in df.columns and (1.0 - df[alias].isna().mean()) == 0:
                    print(f"DÚVIDA_BLOQUEANTE: Cobertura nula (0%) para o macro '{alias}' no tabelão '{filename}'. Abortando."); return

        # Gravação
        final_filepath = get_unique_filepath(os.path.join(SILVER_PATH, filename))
        try:
            if final_filepath.endswith('.parquet'):
                df.to_parquet(final_filepath, engine="pyarrow", index=True, compression="snappy")
            else:
                df.to_csv(final_filepath, index=False)

            # Validação pós-gravação
            if not os.path.exists(final_filepath) or os.path.getsize(final_filepath) == 0:
                print(f"DÚVIDA_BLOQUEANTE: Falha na validação pós-escrita para '{final_filepath}'. Arquivo ausente ou vazio. Abortando."); return

            size_bytes = os.path.getsize(final_filepath)
            print(f"SUCESSO: Salvo em: {final_filepath} (size={size_bytes} bytes)")
            saved_files.append(final_filepath)
        except Exception as e:
            print(f"DÚVIDA_BLOQUEANTE: Exceção durante a escrita de '{final_filepath}': {e}\n{traceback.format_exc()}"); return

        # Relatório
        print(f"  - Shape: {df.shape}")
        if is_tabelao:
            print(f"  - Intervalo: {df.index.min().date()} -> {df.index.max().date()}")
            print("  - Top-5 NaNs:")
            for col, count in df.isna().sum().sort_values(ascending=False).head(5).items(): print(f"    - {col}: {count} ({count/df.shape[0]:.2%})")
            print("  - Cobertura macros:")
            for alias in MACRO_ALIASES:
                if alias in df.columns: print(f"    - {alias}: {1.0 - df[alias].isna().mean():.2%}")

    # =========================================================================
    # PARTE B — Manifesto Silver
    # =========================================================================
    print("\n" + "="*25 + " PARTE B: Manifesto Silver " + "="*28)

    # 1. Descoberta de Artefatos em 02_silver/
    try:
        silver_files = [os.path.join(SILVER_PATH, f) for f in os.listdir(SILVER_PATH)]
        print(f"Iniciando varredura de {len(silver_files)} artefatos em {SILVER_PATH}")
    except Exception as e:
        print(f"DÚVIDA_BLOQUEANTE: Erro ao varrer o diretório Silver '{SILVER_PATH}': {e}\n{traceback.format_exc()}"); return

    # 2. Geração dos Dados do Manifesto
    manifest_data = []
    for f_path in silver_files:
        try:
            f_name = os.path.basename(f_path)
            f_type = 'tabelao' if 'silver_' in f_name and f_name.endswith('.parquet') else ('dim' if 'dim_symbols' in f_name else ('map' if 'map_symbol' in f_name else 'other'))

            stat = os.stat(f_path)
            head_hash, tail_hash = calculate_partial_hashes(f_path)

            df = pd.read_parquet(f_path) if f_path.endswith('.parquet') else pd.read_csv(f_path)
            if df.shape[0] == 0 and stat.st_size > 0 and f_type == 'tabelao':
                print(f"DÚVIDA_BLOQUEANTE: Inconsistência no artefato '{f_name}'. Tem tamanho > 0 mas foi lido com 0 linhas. Abortando."); return

            d_min, d_max = ("N/A", "N/A")
            if isinstance(df.index, pd.DatetimeIndex): d_min, d_max = df.index.min().date(), df.index.max().date()

            rec = {
                'artifact_name': f_name, 'artifact_type': f_type, 'file_path': f_path,
                'rows_total': df.shape[0], 'cols_total': df.shape[1],
                'date_min': d_min, 'date_max': d_max,
                'created_ts': datetime.fromtimestamp(stat.st_ctime).isoformat(),
                'modified_ts': datetime.fromtimestamp(stat.st_mtime).isoformat(),
                'file_size_bytes': stat.st_size,
                'sha256_head20': head_hash, 'sha256_tail20': tail_hash,
                'columns_sample_head': ', '.join(df.columns[:10]) if df.shape[1] > 0 else '',
                'non_na_ratio_mean': None, 'non_na_ratio_min': None
            }
            if f_type == 'tabelao' and not df.empty:
                ratios = 1.0 - df.isna().mean()
                rec['non_na_ratio_mean'] = round(ratios.mean(), 4)
                rec['non_na_ratio_min'] = round(ratios.min(), 4)
            manifest_data.append(rec)
        except Exception as e:
            print(f"DÚVIDA_BLOQUEANTE: Exceção ao processar '{f_path}' para o manifesto: {e}\n{traceback.format_exc()}"); return

    # 3. Gravação e Relatório do Manifesto
    manifest_df = pd.DataFrame(manifest_data)
    manifest_df = manifest_df[list(rec.keys())] # Garantir ordem das colunas

    manifest_filepath = get_unique_filepath(os.path.join(SILVER_PATH, f"silver_manifest_{TODAY_SUFFIX}.csv"))
    manifest_df.to_csv(manifest_filepath, index=False)

    if not os.path.exists(manifest_filepath) or os.path.getsize(manifest_filepath) == 0:
        print(f"DÚVIDA_BLOQUEANTE: Falha na validação pós-escrita para o manifesto '{manifest_filepath}'. Abortando."); return

    print("\n--- Relatório Final do Manifesto ---")
    print(f"Total de arquivos inventariados: {len(manifest_df)}")
    print("Amostra do manifesto (head 5):")
    print(manifest_df.head())
    print("-" * 50)
    print(f"SUCESSO: Manifesto salvo em: {manifest_filepath}")

    print("\n--- Lista Final de Arquivos Gravados Nesta Execução ---")
    for path in saved_files + [manifest_filepath]:
        print(f"- {path}")

    print("\n" + "="*50)
    print("--- EXECUÇÃO CONCLUÍDA COM SUCESSO ---")

# Invocar a função principal
execute_silver_persist_and_manifest_v1()

--- INICIANDO EXECUÇÃO: SILVER_PERSIST_E_MANIFEST_V1 ---
Modo de Execução: PERSIST (Gravação Real)
--------------------------------------------------
Mounted at /content/drive
Caminhos validados.
RAW: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/
SILVER: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/02_silver/

========================= PARTE A: Persistência Silver =========================
Encontrados 31 arquivos Parquet na camada RAW.

--- Construindo Calendário Canônico B3 ---
Calendário Canônico B3 construído: 3409 dias, de 2012-01-02 a 2025-09-19.

--- Lendo, Alinhando e Montando Artefatos Silver ---

--- Gravando e Validando Artefatos Silver ---

--------------------
Processando: silver_close_20250923.parquet
SUCESSO: Salvo em: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/02_silver/silver_close_20250923_v2.parquet (size=561709 bytes)
  - Shape: (3409, 31)
  - Intervalo: 2012-01-02 -> 2025-09-19
  - Top-5 NaNs:
 